In [17]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import xgboost as xgb
import numpy as np
import os 
from pathlib import Path
import pickle 
import pandas as pd
from sklearn.metrics import roc_auc_score
import datetime
import warnings
warnings.filterwarnings('ignore')

from google.colab import drive
drive.mount('/content/drive')

WORK_PATH = "/content/drive/My Drive/research/internship/fraud-detection"
%cd /content/drive/My Drive/research/internship/fraud-detection/

# Load processed Dataset which unnessary columns are deleted 
# the processed data has the same number of rows, but with smaller number of columns
pd00 = pd.read_csv(WORK_PATH + "/data/processed_data.csv")
print(pd00.shape)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/research/internship/fraud-detection
(786363, 26)


In [18]:
print( pd00.columns.tolist())

['accountNumber', 'customerId', 'creditLimit', 'availableMoney', 'transactionDateTime', 'transactionAmount', 'merchantName', 'posEntryMode', 'posConditionCode', 'merchantCategoryCode', 'currentExpDate', 'accountOpenDate', 'dateOfLastAddressChange', 'cardCVV', 'enteredCVV', 'cardLast4Digits', 'transactionType', 'currentBalance', 'cardPresent', 'isFraud', 'date', 'tvt_code_0', 'tvt_code_1', 'tvt_code_2', 'tvt_code_3', 'tvt_code_4']


***I. Identify reversed transactions***

In [4]:
main_cols = ['accountNumber','transactionDateTime', 'merchantName','transactionAmount','transactionType','currentBalance', 'creditLimit','cardPresent'] 
pd00 = pd00.sort_values( by = ['accountNumber','transactionDateTime'])[main_cols]
pd01 = pd00[(pd00['transactionType'] =='REVERSAL') & (pd00['transactionType'].shift(-1) == "PURCHASE")]
pd01.shape

(19382, 8)

In [5]:
pd01.head(4)

,accountNumber,transactionDateTime,merchantName,transactionAmount,transactionType,currentBalance,creditLimit,cardPresent
541964,100088067,2016-11-20T08:00:04,AMC #79863,22.32,REVERSAL,188.12,50000,True
87920,100328049,2016-01-15T20:36:18,Lyft,43.74,REVERSAL,394.90,5000,False
87936,100328049,2016-03-26T17:35:09,McDonalds #423357,284.97,REVERSAL,1207.91,5000,True
151499,100737756,2016-06-06T22:56:52,34th BBQ #436606,93.67,REVERSAL,130.49,250,True


In [6]:
# Total number of reversed transaction on this dataset is around 19382
# Total Amount money of these transactions is:
pd01['transactionAmount'].sum()


2693577.98

In [7]:
print(np.percentile( pd00['transactionAmount'].values, [25, 50, 75]))
print(np.percentile( pd01['transactionAmount'].values, [25, 50, 75]))

[ 33.65  87.9  191.48]
[ 33.5225  92.025  193.29  ]


In [8]:
print(np.percentile( pd00['creditLimit'].values, [25, 50, 75]))
print(np.percentile( pd01['creditLimit'].values, [25, 50, 75]))

[ 5000.  7500. 15000.]
[ 5000.  7500. 15000.]


In [9]:
print(np.percentile( pd00['currentBalance'].values, [25, 50, 75]))
print(np.percentile( pd01['currentBalance'].values, [25, 50, 75]))

[ 689.91  2451.76  5291.095]
[ 885.7625 2807.615  5823.7975]


In [10]:
pd01['merchantName'].value_counts(normalize=True)

Lyft                      0.034155
Uber                      0.034104
alibaba.com               0.024456
oldnavy.com               0.024095
ebay.com                  0.024043
                            ...   
CVS #923193               0.000052
WSC #697470               0.000052
Taco Bell #631524         0.000052
Planet Fitness #835799    0.000052
Hardee's #533765          0.000052
Name: merchantName, Length: 1903, dtype: float64

In [11]:
pd00['merchantName'].value_counts(normalize=True)

Uber                        0.032571
Lyft                        0.032457
oldnavy.com                 0.021608
staples.com                 0.021593
alibaba.com                 0.021566
                              ...   
Golds Gym #956354           0.000003
Boost Mobile #104815        0.000003
TMobile Wireless #602341    0.000001
EZ Wireless #149871         0.000001
Curves #849125              0.000001
Name: merchantName, Length: 2490, dtype: float64

In [19]:
# It seems the reversed transaction and normal transaction looks quite similar except Balance column


**II. Identify Multi-swipe transactions**

In [31]:
# Extract date, time, hour first
def get_second_in_year(trans_date):
    """
    get_second number based on date
    return an integer number between 0 and 3600*24*365
    
    """
    ymd = trans_date.split("T")[0]
    hour = trans_date.split("T")[1].split(":")[0]
    min =  trans_date.split("T")[1].split(":")[1]
    second = trans_date.split("T")[1].split(":")[1]
    y,m,d  = ymd.split("-")
    start_time = datetime.datetime(2016,1,1,0,0,0)
    end_time = datetime.datetime(int(y), int(m),int(d),int(hour), int(min), int(second))
    
    return abs(start_time - end_time).total_seconds() 


pd00['date'] = pd00['transactionDateTime'].apply(lambda x: x.split("T")[0]).astype(str)
pd00['year'] = pd00['date'].apply(lambda x: str(x).split("-")[0])
pd00['month'] = pd00['date'].apply(lambda x: str(x).split("-")[1])
pd00['day'] = pd00['date'].apply(lambda x: str(x).split("-")[2])
pd00['time'] = pd00['transactionDateTime'].apply(lambda x: x.split("T")[1]).astype(str)
pd00['hour'] = pd00['time'].apply(lambda x: x.split(":")[0])
pd00['second_in_year'] = pd00['transactionDateTime'].apply(get_second_in_year)
pd00['enter_correct'] = (pd00['cardCVV'] == pd00['enteredCVV']).astype(int)
pd00['enter_correct'].value_counts(normalize = True)

1    0.991079
0    0.008921
Name: enter_correct, dtype: float64

In [33]:
sub_cols = ['accountNumber', 'merchantName','transactionAmount','date', 'hour']
# Intuition multi-swipe transactions will have same account number, merchantNamt, transaction amount, date and hour
# we first identiy set  of rows of the same these values 
pd02 = pd00[pd00.duplicated(sub_cols, keep = False)]
pd02.head(5)

,accountNumber,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,posEntryMode,posConditionCode,merchantCategoryCode,currentExpDate,accountOpenDate,dateOfLastAddressChange,cardCVV,enteredCVV,cardLast4Digits,transactionType,currentBalance,cardPresent,isFraud,date,tvt_code_0,tvt_code_1,tvt_code_2,tvt_code_3,tvt_code_4,year,month,day,time,hour,second_in_year,enter_correct
38,574788567,574788567,2500,2416.11,2016-05-24T01:35:33,215.13,Convenient Tire,9.0,1.0,auto,10/2021,2015-10-13,2015-10-13,206,206,8522,PURCHASE,83.89,True,0,2016-05-24,train,train,val,train,val,2016,05,24,01:35:33,01,12447335.0,1
39,574788567,574788567,2500,2200.98,2016-05-24T01:38:03,215.13,Convenient Tire,9.0,1.0,auto,10/2021,2015-10-13,2015-10-13,206,206,8522,REVERSAL,299.02,True,0,2016-05-24,train,train,train,train,val,2016,05,24,01:38:03,01,12447518.0,1
132,984504651,984504651,50000,46818.15,2016-01-16T09:52:30,450.74,Planet Fitness #849960,5.0,1.0,health,08/2028,2015-07-27,2015-07-27,640,640,8332,PURCHASE,3181.85,False,0,2016-01-16,train,train,train,train,train,2016,01,16,09:52:30,09,1331572.0,1
133,984504651,984504651,50000,46367.41,2016-01-16T09:53:15,450.74,Planet Fitness #849960,5.0,1.0,health,08/2028,2015-07-27,2015-07-27,640,640,8332,REVERSAL,3632.59,False,0,2016-01-16,train,train,val,train,train,2016,01,16,09:53:15,09,1331633.0,1
199,984504651,984504651,50000,36487.41,2016-02-12T05:08:23,34.92,Golds Gym #747443,9.0,1.0,health,03/2028,2015-07-27,2015-07-27,640,640,8332,PURCHASE,13512.59,False,0,2016-02-12,train,train,train,val,train,2016,02,12,05:08:23,05,3647288.0,1


In [34]:
pd02['merchantName'].value_counts(normalize= True)

Uber                     0.036958
Lyft                     0.033609
apple.com                0.025493
alibaba.com              0.024626
gap.com                  0.024468
                           ...   
Auntie Anne's #938796    0.000079
Powerlifting #918010     0.000079
Walgreens #149062        0.000079
Starbucks #102806        0.000079
Popeyes #972423          0.000079
Name: merchantName, Length: 1670, dtype: float64

In [35]:
print(np.percentile(pd02['transactionAmount'].values, [25, 50, 75]))

[ 37.13   98.465 200.5  ]


In [36]:
pd02['enter_correct'].value_counts(normalize = True)

1    0.99145
0    0.00855
Name: enter_correct, dtype: float64

In [37]:
pd00['enter_correct'].value_counts(normalize = True)

1    0.991079
0    0.008921
Name: enter_correct, dtype: float64

In [42]:
pd02['month'].value_counts(normalize = True)

08    0.088652
10    0.086682
05    0.086013
01    0.085500
07    0.085264
11    0.085067
03    0.084358
12    0.082151
09    0.081009
04    0.080615
06    0.077935
02    0.076753
Name: month, dtype: float64

In [43]:
pd00['month'].value_counts(normalize = True)

10    0.088543
12    0.087715
08    0.086638
11    0.086597
07    0.085405
09    0.084919
05    0.083535
06    0.082322
03    0.081295
04    0.079649
01    0.078300
02    0.075082
Name: month, dtype: float64

In [23]:
# we define multi-swipe can happen at most in 300 seconds = 5 minutes
# pd03 is 
pd03 = pd02[abs( pd02['second_in_year'] -pd02['second_in_year'].shift(1)) <300 ]
print(pd03.shape)

(13392, 32)


In [16]:
pd00 = pd00.drop(pd03.index )
print(pd00.shape)
pd00 = pd.concat([pd00, pd03])
print(pd00.shape)

In [44]:
pd00.shape

(786363, 33)